# 财务数据

**由于数据采集原因，对于没有财务公告日期的，设置为报告期后45天。即模拟回测时钟日期超过报告截止日期，才可读取到该季度的财务报告**

In [1]:
from zipline.pipeline.fundamentals import Fundamentals
from zipline.research import run_pipeline, select_output_by
from zipline.pipeline import Pipeline

**所有数据集科目编码规则都相同，A001->A(N)(连续三位整数）**

## 资产负债表

### 科目含义

In [2]:
# 查询列所对应的科目全称
Fundamentals.balancesheet_col_name('A001')

'货币资金'

In [3]:
Fundamentals.balancesheet_col_name('A108')

'负债和所有者权益或股东权益总计'

### 科目编码

In [4]:
# 模糊查询科目编码
Fundamentals.balancesheet_col_code('投资')

{'A028': '持有至到期投资',
 'A030': '长期股权投资',
 'A031': '其他长期投资',
 'A032': '投资性房地产',
 'A101': '未确定的投资损失'}

### 运行

In [5]:
def make_pipeline():

    latest_fund = Fundamentals.balance_sheet.A001.latest
    total_asset = Fundamentals.balance_sheet.A108.latest

    return Pipeline(columns={
        '货币资金': latest_fund,
        '负债和所有者权益或股东权益总计': total_asset
    })

In [6]:
# 此时返回2017年三季度数据
result = run_pipeline(make_pipeline(), '2017-12-29', '2017-12-31')
result.head()

负债和所有者权益或股东权益总计        货币资金
2017-12-29 00:00:00+00:00 平安银行(000001)      313748100.0  27970600.0
                          万 科Ａ(000002)      101838199.0   9435263.0
                          国农科技(000004)          24121.0     11831.0
                          世纪星源(000005)         279287.0      5850.0
                          深振业Ａ(000006)        1322269.0    387660.0

### `截止日期`(`asof_date`) & `知晓日期`(`timestamp`)

万科A(000002)于2018年3月27日披露年度，回测时钟当日即知晓2017年年报。此前回测时钟日期，最后的数据为2017年三季度报告。(注意`pipeline`特点，只能查询尾部窗口数据，28日实际获取的是28日前的尾部窗口，在此即27日）

In [7]:
# 此时返回2017年年度数据
result = run_pipeline(make_pipeline(), '2018-3-25', '2018-3-28')
select_output_by(result, stock_codes=['000002'])

,,负债和所有者权益或股东权益总计,货币资金
2018-03-26 00:00:00+00:00,万 科Ａ(000002),101838199.0,9435263.0
2018-03-27 00:00:00+00:00,万 科Ａ(000002),101838199.0,9435263.0
2018-03-28 00:00:00+00:00,万 科Ａ(000002),116534692.0,17412101.0


美的集团（000333)于2018年3月31日公布年报，注意观察以下二者的输出结果

In [8]:
select_output_by(result, stock_codes=['000333','000002'])

负债和所有者权益或股东权益总计        货币资金
2018-03-26 00:00:00+00:00 万 科Ａ(000002)      101838199.0   9435263.0
                          美的集团(000333)       24218328.0   4165744.0
2018-03-27 00:00:00+00:00 万 科Ａ(000002)      101838199.0   9435263.0
                          美的集团(000333)       24218328.0   4165744.0
2018-03-28 00:00:00+00:00 万 科Ａ(000002)      116534692.0  17412101.0
                          美的集团(000333)       24218328.0   4165744.0

In [9]:
result = run_pipeline(make_pipeline(), '2018-3-28', '2018-4-2')
select_output_by(result, stock_codes=['000333','000002'])

负债和所有者权益或股东权益总计        货币资金
2018-03-28 00:00:00+00:00 万 科Ａ(000002)      116534692.0  17412101.0
                          美的集团(000333)       24218328.0   4165744.0
2018-03-29 00:00:00+00:00 万 科Ａ(000002)      116534692.0  17412101.0
                          美的集团(000333)       24218328.0   4165744.0
2018-03-30 00:00:00+00:00 万 科Ａ(000002)      116534692.0  17412101.0
                          美的集团(000333)       24218328.0   4165744.0
2018-04-02 00:00:00+00:00 万 科Ａ(000002)      116534692.0  17412101.0
                          美的集团(000333)       24810686.0   4827420.0

美的集团公布年报后的第一个交易日为4月2日，上一交易日为3月30日。回测时钟日期4月2日尾部1个窗口，其货币资金为年报公布数4827420元。

这与实际情形一致，您不可能预知，只能通过公开发布获取信息。

## 利润表

In [10]:
Fundamentals.profit_col_name('A001')

'营业总收入'

In [11]:
Fundamentals.profit_col_code('每股')

{'A044': '基本每股收益', 'A045': '稀释每股收益'}

In [12]:
def make_pipeline():

    eps = Fundamentals.profit_statement.A044.latest
    income = Fundamentals.balance_sheet.A001.latest

    return Pipeline(columns={
        '基本每股收益': eps,
        '营业总收入': income
    })

In [13]:
result = run_pipeline(make_pipeline(), '2016-04-10', '2018-04-10')
result.head()

基本每股收益       营业总收入
2016-04-11 00:00:00+00:00 平安银行(000001)    1.56  29171500.0
                          万 科Ａ(000002)    1.64   5318038.0
                          国农科技(000004)    0.03      3571.0
                          世纪星源(000005)   -0.03      3282.0
                          深振业Ａ(000006)    0.31    177870.0

## 现金流量表

In [14]:
Fundamentals.cashflow_col_name('A021')

'支付给职工以及为职工支付的现金'

In [15]:
Fundamentals.cashflow_col_code('折旧')

{'A061': '固定资产折旧及油气资产折耗及生产性物资折旧'}

In [16]:
def make_pipeline():

    return Pipeline(columns={
        '支付给职工以及为职工支付的现金': Fundamentals.cash_flow.A021.latest,
        '固定资产折旧及油气资产折耗及生产性物资折旧': Fundamentals.cash_flow.A061.latest
    })

In [17]:
result = run_pipeline(make_pipeline(), '2018-04-01', '2018-04-10')
select_output_by(result, stock_codes=['000333','600076'])

固定资产折旧及油气资产折耗及生产性物资折旧  支付给职工以及为职工支付的现金
2018-04-02 00:00:00+00:00 美的集团(000333)               669588.0        2274054.0
                          康欣新材(600076)                 6494.0           9152.0
2018-04-03 00:00:00+00:00 美的集团(000333)               669588.0        2274054.0
                          康欣新材(600076)                 6494.0           9152.0
2018-04-04 00:00:00+00:00 美的集团(000333)               669588.0        2274054.0
                          康欣新材(600076)                 6494.0           9152.0
2018-04-09 00:00:00+00:00 美的集团(000333)               669588.0        2274054.0
                          康欣新材(600076)                 6494.0           9152.0
2018-04-10 00:00:00+00:00 美的集团(000333)               669588.0        2274054.0
                          康欣新材(600076)                 6494.0           9152.0

### 财务指标

In [18]:
Fundamentals.key_financial_indicator_col_name('A009')

'利润总额'

In [19]:
Fundamentals.earnings_ratio_col_name('A009')

'股本报酬率'

In [20]:
Fundamentals.growth_ratio_col_name('A002')

'净利润增长率'

In [21]:
def make_pipeline():

    return Pipeline(
        columns={
            '利润总额': Fundamentals.key_financial_indicator.A009.latest,
            '股本报酬率': Fundamentals.cash_flow.A009.latest,
            '净利润增长率': Fundamentals.growth_ratio.A002.latest,
        })

In [22]:
result = run_pipeline(make_pipeline(), '2018-04-01', '2018-04-10')
result.tail(10)

净利润增长率      利润总额  股本报酬率
2018-04-10 00:00:00+00:00 康德莱(603987)    19.36   12934.0    NaN
                          中电电机(603988)   -2.10    2836.0    NaN
                          艾华集团(603989)   12.40   26330.0    NaN
                          麦迪科技(603990)   13.06    5552.0    NaN
                          至正股份(603991)    1.33    2417.0    NaN
                          洛阳钼业(603993)  252.77  538182.0    NaN
                          中新科技(603996)   14.95    8703.0    NaN
                          继峰股份(603997)   24.28   26339.0    NaN
                          方盛制药(603998)   30.79    9472.0    NaN
                          读者传媒(603999)  -15.74    5171.0    NaN